In [1]:
from huggingface_hub import login

login()

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, AutoModelForCausalLM
from tokenization_small100 import SMALL100Tokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained("crumb/fake-gpt-j-17m")
model = AutoModelForCausalLM.from_pretrained("crumb/fake-gpt-j-17m")

tokenizer.pad_token = tokenizer.eos_token

In [20]:
tokens = tokenizer("Who are you?", return_tensors="pt")
generated_tokens = model.generate(**tokens)
text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [21]:
text

['Who are you? Ereld oushe owhon ouethe ndce ous; ous']

In [25]:
import pandas as pd

parallel_corpora = pd.read_csv("parallel_corpora.csv")

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="en-to-tl",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [27]:
from transformers import Trainer
import datasets
import numpy as np

import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

corpora = parallel_corpora[
    (parallel_corpora['language1'] == 'english') 
     & (parallel_corpora['language2'] == 'filipino')
][['language1_text', 'language2_text']].sample(frac=0.8/10, random_state=42)

# Split the dataframe into train / eval using indices (avoid DataFrame.isin misuse)
train_df = corpora.sample(frac=2/3, random_state=42)
eval_df = corpora.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

# Convert pandas DataFrames to `datasets.Dataset` objects expected by Trainer
train_dataset = datasets.Dataset.from_pandas(train_df)
eval_dataset = datasets.Dataset.from_pandas(eval_df)

# Preprocess: tokenize source and target text columns into model inputs and labels
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['language1_text']
    targets = examples['language2_text']
    # Tokenize inputs
    model_inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=max_input_length)
    # Tokenize targets (labels) - use the tokenizer's target mode if available
    try:
        with tokenizer.as_target_tokenizer():
            labels_tokenized = tokenizer(targets, padding='max_length', truncation=True, max_length=max_target_length)
    except AttributeError:
        labels_tokenized = tokenizer(targets, padding='max_length', truncation=True, max_length=max_target_length)
    labels = labels_tokenized['input_ids']
    # Replace pad token id's in labels by -100 so they are ignored by the loss
    if getattr(tokenizer, 'eos_token_id', None) is not None:
        labels = [[(tok if tok != tokenizer.pad_token_id else -100) for tok in seq] for seq in labels]
    model_inputs['labels'] = labels
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['language1_text', 'language2_text'])
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=['language1_text', 'language2_text'])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

c:\Users\rek\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/778 [00:00<?, ? examples/s]

C:\Users\rek\AppData\Local\Temp\ipykernel_27744\3675179577.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
